In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
import numpy as np
from matplotlib import pyplot as plt
import torch
from sparse_causal_model_learner_rl.sacred_gin_tune.sacred_wrapper import load_config_files
from sparse_causal_model_learner_rl.learners.rl_learner import CausalModelLearnerRL
from sparse_causal_model_learner_rl.config import Config
from keychest.features_xy import arr_to_dict, dict_to_arr, obs_features_handcoded
from keychest.gofa_model import manual_model_features
from sparse_causal_model_learner_rl.trainable.gumbel_switch import LearnableSwitch, WithInputSwitch
from tqdm.auto import tqdm
from torch import nn
import gin

In [2]:
p = LearnableSwitch((10, ))

In [3]:
list(p.named_parameters())

[('logits',
  Parameter containing:
  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True))]

In [4]:
loss = torch.mean(p.sample_mask(method='gumbel'))

In [5]:
p(torch.randn(4, 10), return_mask=True)

tensor([[0., 1., 1., 1., 0., 0., 0., 1., 1., 0.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1., 0., 1., 1.],
        [0., 1., 1., 0., 1., 0., 1., 0., 0., 1.]], grad_fn=<SelectBackward>)

In [6]:
p.logits.grad = None

In [7]:
#loss.backward()
loss = torch.mean(p(torch.randn(4, 10)))

In [8]:
loss.backward()

In [9]:
p.logits.grad

tensor([[ 3.9175e-03, -1.4350e-02,  7.7669e-03, -8.6792e-03,  8.4719e-03,
         -2.0388e-03,  1.1761e-05, -1.2061e-02, -3.9041e-03, -6.3815e-03],
        [-3.9175e-03,  1.4350e-02, -7.7669e-03,  8.6792e-03, -8.4719e-03,
          2.0388e-03, -1.1761e-05,  1.2061e-02,  3.9041e-03,  6.3815e-03]])

In [10]:
p.sparsify_me()

[('proba_on',
  tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
          0.5000], grad_fn=<SelectBackward>))]

In [11]:
p.softmaxed()

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000], grad_fn=<SelectBackward>)

In [12]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__()
        self.z = nn.Linear(10, 1)
    def forward(self, x):
        return self.z(x)

In [13]:
ws = WithInputSwitch(MyModel, input_shape=(10,))

In [14]:
list(ws.named_parameters())

[('switch.logits',
  Parameter containing:
  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)),
 ('model.z.weight',
  Parameter containing:
  tensor([[-0.1271, -0.2931, -0.1677,  0.0235,  0.0376, -0.0605, -0.1132,  0.3012,
           -0.0368, -0.2281]], requires_grad=True)),
 ('model.z.bias',
  Parameter containing:
  tensor([-0.0094], requires_grad=True))]

In [15]:
ws.sparsify_me()

[('proba_on',
  tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
          0.5000], grad_fn=<SelectBackward>))]

In [16]:
ws(torch.randn(5, 10))

tensor([[-0.0127],
        [-1.1530],
        [-0.0380],
        [-0.2747],
        [-0.1677]], grad_fn=<AddmmBackward>)

In [21]:
ws.switch(torch.ones(5, 10))

tensor([[0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 1., 1., 1., 0., 1.],
        [1., 1., 0., 0., 1., 0., 1., 1., 0., 1.],
        [1., 0., 0., 0., 1., 0., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1., 1., 1., 0., 1.]], grad_fn=<MulBackward0>)

In [18]:
?torch.repeat

Object `torch.repeat` not found.
